In [119]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex

In [120]:
coupon_list_train = pd.read_csv('input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('input/coupon_list_test.csv')
user_list = pd.read_csv('input/user_list.csv')
coupon_purchases_train = pd.read_csv("input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('input/coupon_visit_train.csv')

In [121]:
coupon_list_train

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
0,グルメ,グルメ,50,3000,1500,2011-07-08 12:00:00,2011-07-09 12:00:00,1,2011-07-10,2011-12-08,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,埼玉県,埼玉,6b263844241eea98c5a97f1335ea82af
1,グルメ,グルメ,51,2080,1000,2011-07-01 12:00:00,2011-07-02 12:00:00,1,2011-07-03,2011-12-04,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,cc031f250e8bad1e24060263b9fc0ddd
2,グルメ,グルメ,50,7000,3500,2011-07-12 12:00:00,2011-07-15 12:00:00,3,2011-07-16,2012-01-11,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,ba5e9b7453ca52ff711635a5d2e8102d
3,グルメ,グルメ,50,3000,1500,2011-07-09 12:00:00,2011-07-11 12:00:00,2,2011-07-12,2011-12-01,...,1.0,0.0,0.0,1.0,1.0,1.0,関東,千葉県,千葉,3e1ffbedca3569f9e8032d401e8cb4e6
4,グルメ,グルメ,50,2000,1000,2011-07-05 12:00:00,2011-07-06 12:00:00,1,2011-07-07,2011-12-30,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,千葉県,千葉,782934b6c815b4030ea204eef7d4a734
5,グルメ,グルメ,50,4000,2000,2011-07-24 12:00:00,2011-07-26 12:00:00,2,2011-07-27,2011-10-30,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,439db4336f84d4e4a13290fc0d7e6c4e
6,グルメ,グルメ,51,3495,1700,2011-09-03 12:00:00,2011-09-05 12:00:00,2,2011-09-07,2012-02-28,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,新宿・高田馬場・中野・吉祥寺,c9144a97ed61cb778414b758c704d828
7,グルメ,グルメ,51,3100,1500,2011-08-02 12:00:00,2011-08-04 12:00:00,2,2011-08-05,2012-01-31,...,1.0,1.0,1.0,0.0,1.0,1.0,関西,京都府,京都,2b00d8dfc1a9fcf58e5014cbfd4a1e48
8,グルメ,グルメ,50,3500,1750,2011-08-11 12:00:00,2011-08-13 12:00:00,2,2011-08-14,2011-11-14,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,恵比寿・目黒・品川,74e3a12659d38ef20378f156cfeaf204
9,グルメ,グルメ,57,10500,4500,2011-09-21 12:00:00,2011-09-24 12:00:00,3,2011-09-26,2012-02-29,...,0.0,0.0,1.0,0.0,0.0,1.0,関東,東京都,銀座・新橋・東京・上野,8a0ab8c5dd5014df8b4860233238ed44


In [122]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash',
                                                 how='inner')

purchased_coupons_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,...,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,1,2012-03-26 23:26:24,千葉,028ac37f96a17e6be2db51ed0884ac5a,dd8a79645e8170db6a91bad1d89b4055,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,1,2012-03-26 13:50:20,新宿・高田馬場・中野・吉祥寺,eb7f867c9f1b14c1a2b1cc47004828ae,658249379aaaf2b2320061e5218c0c2c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
3,1,2012-03-28 22:46:58,和歌山,fddda2bdaba87fc9b86d97e908152014,1c4aad19f033d97dc163b481116de322,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
4,1,2012-03-29 15:09:51,宮崎,6659c8fdb5f66b4013613ff770f91db7,e78ae0887c4a9dce91ff0d2a45a9f72c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
5,1,2012-03-29 12:42:37,千葉,be5af03efe7e4bc2f1e7e8317531cd74,39cabe60f73ea8cad96d7f895ea944ee,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
6,1,2012-03-28 22:45:16,和歌山,aae8c29627af3489c530f141eb02a3a1,fca1e37548c9ca5415a9850a1b9b2c28,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
7,1,2012-03-29 18:10:28,栃木,ea4cf60407a03246be6c588c12542780,3511c63ae9ce45fc3b5cf5054956d5a4,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
8,1,2012-03-28 00:05:32,栃木,448cec2c62179725c9ac0d93006074d0,77b92939627690028e22834d7920bf4e,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
9,2,2012-03-27 11:47:59,栃木,f8f426b0f2beb05978e856a727febac8,1ce49ceb6f55d3b3ed3180f811ec3ffb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [123]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [124]:
purchased_coupons_train = purchased_coupons_train[features]

In [125]:
coupon_list_test['USER_ID_hash'] = 'dummyuser'

In [126]:
coupon_list_test = coupon_list_test[features]

In [127]:
combined = pd.concat([purchased_coupons_train, coupon_list_test], axis=0)

In [128]:
combined['DISCOUNT_PRICE'] = 1 / np.log10(combined['DISCOUNT_PRICE'])
combined['PRICE_RATE'] = (combined['PRICE_RATE'] / 100) ** 2
features.extend(['DISCOUNT_PRICE', 'PRICE_RATE'])

In [129]:
categoricals = ['GENRE_NAME', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
                'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
                'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
combined_categoricals = combined[categoricals]
combined_categoricals = pd.get_dummies(combined_categoricals,
                                    dummy_na=False)

In [130]:
continuous = list(set(features) - set(categoricals))
combined = pd.concat([combined[continuous], combined_categoricals], axis=1)

In [131]:
NAN_SUBSTITUTION_VALUE = 1
combined = combined.fillna(NAN_SUBSTITUTION_VALUE)

In [132]:
train = combined[combined['USER_ID_hash'] != 'dummyuser']
test = combined[combined['USER_ID_hash'] == 'dummyuser']
test.drop('USER_ID_hash', inplace=True, axis=1)

In [133]:
train_dropped_coupons = train.drop('COUPON_ID_hash', axis=1)
user_profiles = train_dropped_coupons.groupby(by='USER_ID_hash').apply(np.mean)

In [135]:
FEATURE_WEIGHTS = {
    'GENRE_NAME': 2,
    'DISCOUNT_PRICE': 2,
    'PRICE_RATE': 0,
    'USABLE_DATE_': 0,
    'large_area_name': 0.5,
    'ken_name': 1,
    'small_area_name': 5
}

In [137]:
def find_appropriate_weight(weights_dict, colname):
    for col, weight in weights_dict.items():
        if col in colname:
            return weight
    raise ValueError

W_values = [find_appropriate_weight(FEATURE_WEIGHTS, colname) for colname in user_profiles.columns]

W = np.diag(W_values)

In [189]:
user_profiles

,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,...,small_area_name_赤坂・六本木・麻布,small_area_name_銀座・新橋・東京・上野,small_area_name_長崎,small_area_name_長野,small_area_name_青森,small_area_name_静岡,small_area_name_香川,small_area_name_高知,small_area_name_鳥取,small_area_name_鹿児島
USER_ID_hash,,,,,,,,,,,,,,,,,,,,,
0000b53e182165208887ba65c079fc21,0.287835,0.384400,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
00035b86e6884589ec8d28fbf2fe7757,0.315141,0.270400,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
0005b1068d5f2b8f2a7c978fcfe1ca06,0.366369,0.664200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
000cc06982785a19e2a2fdb40b1c9d59,0.332673,0.459695,0.904762,1.000000,1.000000,1.000000,1.095238,1.095238,0.952381,0.952381,...,0.000000,0.142857,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
0013518e41c416cd6a181d277dd8ca0b,0.390656,0.546875,1.000000,1.000000,1.000000,1.000000,1.000000,1.250000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
001acdee812a18acfd7509172bed5700,0.349592,0.437362,1.000000,1.000000,1.000000,0.923077,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.076923,0.0,0.0,0.0
001fd7876e3aa29393537c6baf308e43,0.307422,0.409600,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
002383753c1e5d6305c8aff6f89e26d6,0.333333,0.313600,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
0025cae7997d25ea5cf8851bb099c798,0.329024,0.374289,1.000000,0.888889,1.111111,1.000000,1.111111,1.000000,0.777778,0.888889,...,0.000000,0.111111,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0


In [139]:
test_only_features = test.drop('COUPON_ID_hash', axis=1)
similarity_scores = np.dot(np.dot(user_profiles, W), test_only_features.T)

In [140]:
coupons_ids = test['COUPON_ID_hash']
index = user_profiles.index
columns = [coupons_ids[i] for i in range(0, similarity_scores.shape[1])]
result_df = pd.DataFrame(index=index, columns=columns, data=similarity_scores)

In [141]:
df_temp = user_profiles.copy()

In [142]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_temp = df_temp.reset_index()
categorical_cols = ['USER_ID_hash']

df_temp[categorical_cols] = df_temp[categorical_cols].apply(lambda col: le.fit_transform(col))
df_temp = df_temp.set_index('USER_ID_hash')

In [192]:
l = df_temp.shape[0]
f = len(df_temp.columns)
t = AnnoyIndex(f, 'angular')

for i in range(l):
    v = [df_temp.values[i][z] for z in range(f)]
    t.add_item(i, v)
    
t.build(100)

True

In [207]:
test_copy = test.copy()
test_copy = test_copy.drop('COUPON_ID_hash', axis=1)
test_copy.loc[0].values

array([0.29156625, 0.2704    , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [210]:
sim = t.get_nns_by_vector(test_copy.loc[0].values, 5)
sim

[6009, 11908, 12632, 13057, 1433]

In [146]:
def get_top10_coupon_hashes_string(row):
    row.sort_values()
    return ' '.join(row.index[-10:][::-1].tolist())

output = result_df.apply(get_top10_coupon_hashes_string, axis=1)


output_df = pd.DataFrame(data={'USER_ID_hash': output.index,
                               'PURCHASED_COUPONS': output.values})
output_df_all_users = pd.merge(user_list, output_df, how='left', on='USER_ID_hash')
output_df_all_users.to_csv('cosine_sim_python.csv', header=True,
                           index=False, columns=['USER_ID_hash', 'PURCHASED_COUPONS'])